In [1]:
import glob
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler

# 폴더 경로 지정
folder_path = r'C:\Users\pc\Desktop\CNN\CIC-AndMal2017'

In [2]:
# 랜섬웨어 클래스별 파일에 있는 샘플 개수를 확인하기 위한 딕셔너리
samples_per_class = {}

# Ransomware 폴더 내의 10개 폴더 경로 리스트 받아오기
ransomware_folder_list = glob.glob(folder_path + '\\Ransomware\\*')

# 모든 파일 로드 및 샘플 개수 확인
for folder in ransomware_folder_list:
    ransomware_class = folder.split('\\')[-1]  # 랜섬웨어 클래스명 추출

    file_paths = glob.glob(folder + '\\*.csv')
    total_samples = 0  # 클래스별 전체 샘플 개수 초기화
    for file_path in file_paths:
        data = pd.read_csv(file_path)
        total_samples += data.shape[0]  # 데이터프레임의 행 수 / 샘플 개수 더하기

    samples_per_class[ransomware_class] = total_samples

# 클래스별 샘플 개수 출력
for ransomware_class, num_samples in samples_per_class.items():
    print(f"{ransomware_class} 클래스의 샘플 개수: {num_samples}")

Charger 클래스의 샘플 개수: 39551
Jisut 클래스의 샘플 개수: 25672
Koler 클래스의 샘플 개수: 44555
Lockerpin 클래스의 샘플 개수: 25307
Pletor 클래스의 샘플 개수: 4715
PornDroid 클래스의 샘플 개수: 46082
RansomBO 클래스의 샘플 개수: 39859
Simplocker 클래스의 샘플 개수: 36340
SVpeng 클래스의 샘플 개수: 54161
WannaLocker 클래스의 샘플 개수: 32701


In [3]:
benign_folder_path = r'C:\Users\pc\Desktop\CNN\CIC-AndMal2017\Benign'

# 'Benign' 클래스의 파일 경로 리스트 받아오기
benign_file_paths = glob.glob(benign_folder_path + '/*.csv')

# 'Benign' 클래스의 데이터프레임 초기화
benign_data = pd.DataFrame()

# 'Benign' 클래스의 모든 파일 로드
for file_path in benign_file_paths:
    data = pd.read_csv(file_path)
    benign_data = pd.concat([benign_data, data], ignore_index=True)

# 'Benign' 클래스에서 45,000개 무작위 샘플 추출
num_samples_benign = 45000
if len(benign_data) > num_samples_benign:
    benign_subset = benign_data.sample(n=num_samples_benign, random_state=42)
else:
    benign_subset = benign_data.copy()

print("benign_subset의 샘플 개수:", len(benign_subset))

benign_subset의 샘플 개수: 45000


In [4]:
# 각 랜섬웨어 클래스별로 4,500개씩 무작위 샘플 추출 (비복원 추출)
num_samples_per_ransomware_class = 4500
all_ransomware_data = []

# Ransomware 폴더 내의 10개 폴더 경로 리스트 받아오기
ransomware_folder_list = glob.glob(folder_path + '\\Ransomware\\*')

for ransomware_folder_path in ransomware_folder_list:
    ransomware_file_paths = glob.glob(ransomware_folder_path + '/*.csv')
    ransomware_data = pd.DataFrame()

    # 각 랜섬웨어 클래스의 모든 파일 로드
    for file_path in ransomware_file_paths:
        data = pd.read_csv(file_path)
        ransomware_data = pd.concat([ransomware_data, data], ignore_index=True)

    # 랜섬웨어 클래스에서 4,500개 무작위 샘플 추출 (비복원 추출)
    if len(ransomware_data) > num_samples_per_ransomware_class:
        ransomware_subset = ransomware_data.sample(n=num_samples_per_ransomware_class, random_state=42, replace=False)
    else:
        ransomware_subset = ransomware_data.copy()
    all_ransomware_data.append(ransomware_subset)

# 모든 랜섬웨어 클래스의 데이터프레임들을 합치기
all_ransomware_data = pd.concat(all_ransomware_data, ignore_index=True)

print("ransomware_subset의 샘플 개수:", len(all_ransomware_data))

ransomware_subset의 샘플 개수: 45000


In [5]:
# 'Benign' 클래스와 랜섬웨어 클래스의 데이터프레임들을 합치기
sub_dataset = pd.concat([benign_subset, all_ransomware_data], ignore_index=True)

print("최종 특성추출 데이터세트의 샘플 개수:", len(sub_dataset))

최종 특성추출 데이터세트의 샘플 개수: 90000


In [6]:
# 타깃 변수 열 이름 확인
target_variable = ' Label'  # 실제 타깃 변수 열 이름으로 수정

# X와 y로 데이터 분할
y = sub_dataset[target_variable]  # 타깃 변수
X = sub_dataset.drop([target_variable], axis=1)  # 타깃 변수 제외한 나머지 특성

# 불필요한 특성 제거 (예시: 'Flow ID', ' Timestamp', ' Source IP', ' Destination IP' 특성 제거)
unnecessary_features = ['Flow ID', ' Timestamp', ' Source IP', ' Destination IP']
X = X.drop(unnecessary_features, axis=1)

# 각 열(feature)에 Min-Max 스케일링 적용
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# chi2를 사용하여 특성 선택 (하위 36개의 특성 선택)
num_features_to_select = 36
selector = SelectKBest(score_func=chi2, k=num_features_to_select)
X_selected = selector.fit_transform(X_scaled, y)

# 선택된 특성들의 인덱스 추출
selected_feature_indices = selector.get_support(indices=True)

# 선택된 특성들의 컬럼명 추출
selected_feature_names = X.columns[selected_feature_indices]

# 최종 선택된 특성들의 데이터프레임 생성
X_final = X[selected_feature_names]

# 결과 출력
print("선택된 특성들의 컬럼명:")
print(selected_feature_names)
print("\n최종 선택된 특성들의 데이터프레임:")
print(X_final.head())

선택된 특성들의 컬럼명:
Index([' Source Port', ' Destination Port', ' Flow Duration',
       'Total Length of Fwd Packets', ' Fwd Packet Length Max',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', ' Flow IAT Mean',
       ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean',
       ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total',
       ' Bwd IAT Max', 'Fwd PSH Flags', 'FIN Flag Count', ' SYN Flag Count',
       ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count',
       ' Avg Bwd Segment Size', ' Subflow Fwd Bytes', 'Init_Win_bytes_forward',
       ' Init_Win_bytes_backward', ' act_data_pkt_fwd', 'Active Mean',
       ' Active Std', ' Active Max', ' Active Min', 'Idle Mean', ' Idle Std',
       ' Idle Max', ' Idle Min'],
      dtype='object')

최종 선택된 특성들의 데이터프레임:
    Source Port   Destination Port   Flow Duration  \
0         38807                 53           95317   
1         49679                 80          258421   
2         36975           

In [7]:
# 특성들의 가중치(Chi-square 통계량) 확인
chi2_scores = selector.scores_[selected_feature_indices]

# 특성들의 가중치를 기준으로 오름차순 정렬
sorted_indices = chi2_scores.argsort()
sorted_features = selected_feature_names[sorted_indices]
sorted_chi2_scores = chi2_scores[sorted_indices]

# 특성들의 가중치와 순위 출력
print("특성들의 순위와 가중치:")
for i, (feature, score) in enumerate(zip(sorted_features, sorted_chi2_scores), 1):
    print(f"{i}. {feature}: {score}")

특성들의 순위와 가중치:
1. Bwd IAT Total: 54.57963310395508
2.  Bwd IAT Max: 55.73380181785601
3.  Fwd IAT Std: 55.972731191232015
4.  Bwd Packet Length Mean: 63.1108563490311
5.  Avg Bwd Segment Size: 63.11085634903111
6.  Fwd Packet Length Max: 67.2275268599415
7.  Fwd IAT Mean: 68.41141305806283
8.  Idle Min: 73.25565940214791
9.  Bwd Packet Length Std: 73.39718419578342
10. Idle Mean: 73.8250603674548
11.  Idle Std: 78.78845887579168
12.  Flow IAT Mean: 78.87444216929649
13.  Flow IAT Max: 82.33798579259249
14.  Fwd IAT Min: 82.58487629530032
15.  Idle Max: 82.99597046558867
16.  Flow IAT Min: 91.92792382528727
17.  Fwd IAT Max: 96.41990946338971
18.  Active Min: 99.60030874564774
19.  Init_Win_bytes_backward: 119.22358007700582
20.  act_data_pkt_fwd: 127.86822541571262
21. FIN Flag Count: 152.16124567478806
22. Total Length of Fwd Packets: 172.717240631204
23.  Subflow Fwd Bytes: 172.717240631204
24.  ACK Flag Count: 176.28001974489948
25. Active Mean: 205.05607524167073
26. Fwd PSH Flags: 

In [8]:
# 'Benign' 클래스의 파일 경로 리스트 받아오기
benign_file_paths = glob.glob(benign_folder_path + '/*.csv')

# 'Benign' 클래스의 데이터프레임 초기화
benign_data = pd.DataFrame()

# 'Benign' 클래스의 모든 파일 로드
for file_path in benign_file_paths:
    data = pd.read_csv(file_path)
    benign_data = pd.concat([benign_data, data], ignore_index=True)

# 'Benign' 클래스에서 160,000개 무작위 샘플 추출
num_samples_benign = 160000
if len(benign_data) > num_samples_benign:
    benign_subset = benign_data.sample(n=num_samples_benign, random_state=42)
else:
    benign_subset = benign_data.copy()

print("train_benign_subset의 샘플 개수:", len(benign_subset))

train_benign_subset의 샘플 개수: 160000


In [9]:
# 각 랜섬웨어 클래스별로 4000개씩 무작위 샘플 추출 (비복원 추출)
num_samples_per_ransomware_class = 4000
all_ransomware_data = []

# Ransomware 폴더 내의 10개 폴더 경로 리스트 받아오기
ransomware_folder_list = glob.glob(folder_path + '\\Ransomware\\*')

for ransomware_folder_path in ransomware_folder_list:
    ransomware_file_paths = glob.glob(ransomware_folder_path + '/*.csv')
    ransomware_data = pd.DataFrame()

    # 각 랜섬웨어 클래스의 모든 파일 로드
    for file_path in ransomware_file_paths:
        data = pd.read_csv(file_path)
        ransomware_data = pd.concat([ransomware_data, data], ignore_index=True)

    # 랜섬웨어 클래스에서 4000개 무작위 샘플 추출 (비복원 추출)
    if len(ransomware_data) > num_samples_per_ransomware_class:
        ransomware_subset = ransomware_data.sample(n=num_samples_per_ransomware_class, random_state=42, replace=False)
    else:
        ransomware_subset = ransomware_data.copy()
    all_ransomware_data.append(ransomware_subset)

# 모든 랜섬웨어 클래스의 데이터프레임들을 합치기
all_ransomware_data_subset = pd.concat(all_ransomware_data, ignore_index=True)

print("ransomware_subset의 샘플 개수:", len(all_ransomware_data_subset))

ransomware_subset의 샘플 개수: 40000


In [10]:
# 'Benign' 클래스와 랜섬웨어 클래스의 데이터프레임들을 합치기
sub_dataset = pd.concat([benign_subset, all_ransomware_data_subset], ignore_index=True)

print("최종 train 데이터세트의 샘플 개수:", len(sub_dataset))

최종 train 데이터세트의 샘플 개수: 200000


In [11]:
from sklearn.preprocessing import MinMaxScaler

# 선택된 36가지 특성에 해당하는 열만 추출
selected_feature_columns = X_final.columns
selected_feature_values = sub_dataset[selected_feature_columns]

# Min-Max 스케일링을 사용하여 선택된 특성들을 0과 1 사이의 값으로 정규화 > normalization으로 쓰기
scaler = MinMaxScaler()
selected_feature_values_normalized = scaler.fit_transform(selected_feature_values)

# 정규화된 특성들을 데이터프레임으로 변환
X_final_normalized = pd.DataFrame(selected_feature_values_normalized, columns=selected_feature_columns)

# 결과 출력
print("정규화된 특성들의 데이터프레임:")
print(X_final_normalized.head())

정규화된 특성들의 데이터프레임:
    Source Port   Destination Port   Flow Duration  \
0      0.592166           0.000811        0.000794   
1      0.758065           0.001224        0.002154   
2      0.564211           0.006779        0.085388   
3      0.739433           0.001224        0.004174   
4      0.720954           0.006779        0.024747   

   Total Length of Fwd Packets   Fwd Packet Length Max  \
0                     0.000004                0.001924   
1                     0.000000                0.000000   
2                     0.000058                0.025503   
3                     0.000101                0.048244   
4                     0.000005                0.001135   

    Bwd Packet Length Mean   Bwd Packet Length Std   Flow IAT Mean  \
0                 0.070548                0.000000        0.000795   
1                 0.000000                0.000000        0.002155   
2                 0.025616                0.062627        0.009495   
3                 0.089212  

In [12]:
# 선택된 36가지 특성을 6x6 크기의 2차원 행렬로 변환
num_rows = 6
num_columns = 6
X_final_reshaped = X_final_normalized.values.reshape(-1, num_rows, num_columns)

# 결과 출력
print("6x6 크기의 2차원 행렬:")
print(X_final_reshaped[0])  # 첫 번째 샘플에 해당하는 6x6 행렬 출력

6x6 크기의 2차원 행렬:
[[5.92165899e-01 8.11005187e-04 7.94301774e-04 4.00783933e-06
  1.92383583e-03 7.05479452e-02]
 [0.00000000e+00 7.94961759e-04 7.94961759e-04 7.95061763e-04
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 7.05479452e-02]
 [4.00783933e-06 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [13]:
from PIL import Image
import os

# 이미지를 저장할 폴더를 생성합니다.
save_folder = r'C:\Users\pc\Desktop\CNN\Train_Image\\'
os.makedirs(os.path.join(save_folder, "Benign"), exist_ok=True)
os.makedirs(os.path.join(save_folder, "Ransomware"), exist_ok=True)

# X_final_reshaped를 이미지로 변환하여 저장합니다.
num_samples = X_final_reshaped.shape[0]
for i in range(num_samples):
    # 1채널 16비트 PNG 이미지 생성
    image_data = X_final_reshaped[i]
    image_data = (image_data * 65535).astype('uint16')  # 16비트로 변환
    image = Image.fromarray(image_data, 'I;16')  # 16비트 단일 채널 이미지로 변환

    # 이미지의 라벨에 따라서 저장 위치 및 파일명 설정
    if i < 160000:
        label = "Benign"
    else:
        label = "Ransomware"

    image_filename = f"image_{i}.png"
    image_filepath = os.path.join(save_folder, label, image_filename)

    # 해당 경로에 이미지가 이미 존재하는 경우 건너뛴다.
    if os.path.exists(image_filepath):
        continue

    image.save(image_filepath)

print("이미지 변환 및 저장이 완료되었습니다.")

이미지 변환 및 저장이 완료되었습니다.


In [14]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 이미지 데이터를 저장한 폴더 경로
image_folder = r'C:\Users\pc\Desktop\CNN\Train_Image\\'

# 이미지 데이터를 불러오고 라벨을 지정합니다.
X_data = []
y_labels = []

for i in range(num_samples):
    if i < 160000:
        label = 0  # Benign 클래스
    else:
        label = 1  # Ransomware 클래스

    image_path = os.path.join(image_folder, f"image_{i}.png")
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # 이미지를 그레이스케일로 불러옵니다.
    X_data.append(image)
    y_labels.append(label)

X_data = np.array(X_data)
y_labels = np.array(y_labels)

In [15]:
# Count the number of samples in each class
num_benign_samples = np.count_nonzero(y_labels == 0)
num_ransomware_samples = np.count_nonzero(y_labels == 1)

print("Number of Benign samples:", num_benign_samples)
print("Number of Ransomware samples:", num_ransomware_samples)

Number of Benign samples: 160000
Number of Ransomware samples: 40000


In [16]:
# 기존에 train 데이터셋으로 사용된 인덱스를 추출
train_benign_indices = benign_subset.index
train_ransomware_indices = all_ransomware_data_subset.index

In [17]:
# 사용되지 않은 benign 데이터에서 추가로 20,000개 추출
remaining_benign_data = benign_data.drop(train_benign_indices)
num_samples_val_benign = 20000
if len(remaining_benign_data) > num_samples_val_benign:
    val_benign_subset = remaining_benign_data.sample(n=num_samples_val_benign, random_state=42)
else:
    val_benign_subset = remaining_benign_data.copy()

print("Validation_benign_subset의 샘플 개수:", len(val_benign_subset))

Validation_benign_subset의 샘플 개수: 20000


In [18]:
num_samples_additional = 400
val_ransomware_subset = []

for ransomware_folder_path in ransomware_folder_list:
    ransomware_file_paths = glob.glob(ransomware_folder_path + '/*.csv')
    ransomware_data = pd.DataFrame()

    # 각 랜섬웨어 클래스의 모든 파일 로드
    for file_path in ransomware_file_paths:
        data = pd.read_csv(file_path)
        ransomware_data = pd.concat([ransomware_data, data], ignore_index=True)

    # 이미 선택된 샘플들 제외
    remaining_data = ransomware_data.drop(train_ransomware_indices, errors='ignore')

    # 400개의 샘플 추출
    if len(remaining_data) > num_samples_additional:
        subset = remaining_data.sample(n=num_samples_additional, random_state=42)
    else:
        subset = remaining_data.copy()
    
    val_ransomware_subset.append(subset)

# 모든 추가된 랜섬웨어 데이터의 데이터프레임들을 합치기
val_ransomware_subset = pd.concat(val_ransomware_subset, ignore_index=True)

print("Validation_ransomware_subset의 샘플 개수:", len(val_ransomware_subset))

Validation_ransomware_subset의 샘플 개수: 1200


In [19]:
# 'Benign' 클래스와 랜섬웨어 클래스의 데이터프레임들을 합치기
val_sub_dataset = pd.concat([val_benign_subset, val_ransomware_subset], ignore_index=True)

print("최종 Validation 데이터세트의 샘플 개수:", len(val_sub_dataset))

최종 Validation 데이터세트의 샘플 개수: 21200


In [20]:
# 'Benign' 클래스와 랜섬웨어 클래스의 데이터프레임들을 합치기
val_sub_dataset = pd.concat([val_benign_subset, val_ransomware_subset], ignore_index=True)

print("최종 Validation 데이터세트의 샘플 개수:", len(val_sub_dataset))

최종 Validation 데이터세트의 샘플 개수: 21200


In [21]:
# 선택된 36가지 특성에 해당하는 열만 추출 (Validation 데이터셋)
val_selected_feature_values = val_sub_dataset[selected_feature_columns]

# Min-Max 스케일링을 사용하여 선택된 특성들을 0과 1 사이의 값으로 정규화 (Validation 데이터셋)
val_selected_feature_values_normalized = scaler.transform(val_selected_feature_values)

# 정규화된 특성들을 데이터프레임으로 변환 (Validation 데이터셋)
X_val_normalized = pd.DataFrame(val_selected_feature_values_normalized, columns=selected_feature_columns)

# 선택된 36가지 특성을 6x6 크기의 2차원 행렬로 변환 (Validation 데이터셋)
X_val_reshaped = X_val_normalized.values.reshape(-1, num_rows, num_columns)


In [22]:
# 이미지를 저장할 폴더를 생성합니다.
val_save_folder = r'C:\Users\pc\Desktop\CNN\Validation_Image\\'
os.makedirs(os.path.join(val_save_folder, "Benign"), exist_ok=True)
os.makedirs(os.path.join(val_save_folder, "Ransomware"), exist_ok=True)

# X_val_reshaped를 이미지로 변환하여 저장합니다.
num_val_samples = X_val_reshaped.shape[0]
for i in range(num_val_samples):
    # 1채널 16비트 PNG 이미지 생성
    image_data = X_val_reshaped[i]
    image_data = (image_data * 65535).astype('uint16')  # 16비트로 변환
    image = Image.fromarray(image_data, 'I;16')  # 16비트 단일 채널 이미지로 변환

    # 이미지의 라벨에 따라서 저장 위치 및 파일명 설정
    if i < num_samples_val_benign:
        label = "Benign"
    else:
        label = "Ransomware"

    image_filename = f"image_{i}.png"
    image.save(os.path.join(val_save_folder, label, image_filename))

print("Validation 이미지 변환 및 저장이 완료되었습니다.")

Validation 이미지 변환 및 저장이 완료되었습니다.


In [23]:
benign_folder_path = r'C:\Users\pc\Desktop\CNN\CIC-AndMal2017\Benign'

# 'Benign' 클래스의 파일 경로 리스트 받아오기
test_benign_file_paths = glob.glob(benign_folder_path + '/*.csv')

# 'Benign' 클래스의 데이터프레임 초기화
test_benign_data = pd.DataFrame()

# 'Benign' 클래스의 모든 파일 로드
for file_path in test_benign_file_paths:
    data = pd.read_csv(file_path)
    test_benign_data = pd.concat([test_benign_data, data], ignore_index=True)

# 'Benign' 클래스에서 20,000개 무작위 샘플 추출 (Test 데이터셋)
num_samples_test_benign = 20000
if len(test_benign_data) > num_samples_test_benign:
    test_benign_subset = test_benign_data.sample(n=num_samples_test_benign, random_state=42)
else:
    test_benign_subset = test_benign_data.copy()

print("test_benign_subset의 샘플 개수:", len(test_benign_subset))

test_benign_subset의 샘플 개수: 20000


In [24]:
# 각 랜섬웨어 클래스별로 400개씩 무작위 샘플 추출 (비복원 추출)
num_samples_per_ransomware_class = 400
all_ransomware_data = []

# Ransomware 폴더 내의 10개 폴더 경로 리스트 받아오기
ransomware_folder_list = glob.glob(folder_path + '\\Ransomware\\*')

for ransomware_folder_path in ransomware_folder_list:
    ransomware_file_paths = glob.glob(ransomware_folder_path + '/*.csv')
    ransomware_data = pd.DataFrame()

    # 각 랜섬웨어 클래스의 모든 파일 로드
    for file_path in ransomware_file_paths:
        data = pd.read_csv(file_path)
        ransomware_data = pd.concat([ransomware_data, data], ignore_index=True)

    # 랜섬웨어 클래스에서 400개 무작위 샘플 추출 (비복원 추출)
    if len(ransomware_data) > num_samples_per_ransomware_class:
        ransomware_subset = ransomware_data.sample(n=num_samples_per_ransomware_class, random_state=42, replace=False)
    else:
        ransomware_subset = ransomware_data.copy()
    all_ransomware_data.append(ransomware_subset)

# 모든 랜섬웨어 클래스의 데이터프레임들을 합치기
all_test_ransomware_data_subset = pd.concat(all_ransomware_data, ignore_index=True)

print("ransomware_subset의 샘플 개수:", len(all_test_ransomware_data_subset))

ransomware_subset의 샘플 개수: 4000


In [25]:
# 'Benign' 클래스와 랜섬웨어 클래스의 데이터프레임들을 합치기 (Test 데이터셋)
test_sub_dataset = pd.concat([test_benign_subset, all_test_ransomware_data_subset], ignore_index=True)

print("최종 Test 서브 데이터세트의 샘플 개수:", len(test_sub_dataset))

최종 Test 서브 데이터세트의 샘플 개수: 24000


In [26]:
# 선택된 36가지 특성에 해당하는 열만 추출 (Test 데이터셋)
test_selected_feature_values = test_sub_dataset[selected_feature_columns]

# Min-Max 스케일링을 사용하여 선택된 특성들을 0과 1 사이의 값으로 정규화 (Test 데이터셋)
test_selected_feature_values_normalized = scaler.transform(test_selected_feature_values)

# 정규화된 특성들을 데이터프레임으로 변환 (Test 데이터셋)
X_test_normalized = pd.DataFrame(test_selected_feature_values_normalized, columns=selected_feature_columns)

# 선택된 36가지 특성을 6x6 크기의 2차원 행렬로 변환 (Test 데이터셋)
X_test_reshaped = X_test_normalized.values.reshape(-1, num_rows, num_columns)

In [27]:
# 이미지를 저장할 폴더를 생성합니다.
test_save_folder = r'C:\Users\pc\Desktop\CNN\Test_Image\\'
os.makedirs(os.path.join(test_save_folder, "Benign"), exist_ok=True)
os.makedirs(os.path.join(test_save_folder, "Ransomware"), exist_ok=True)

# X_test_reshaped를 이미지로 변환하여 저장합니다.
num_test_samples = X_test_reshaped.shape[0]
for i in range(num_test_samples):
    # 이미지의 라벨에 따라서 저장 위치 및 파일명 설정
    if i < num_samples_test_benign:
        label = "Benign"
    else:
        label = "Ransomware"

    image_filename = f"image_{i}.png"
    image_path = os.path.join(test_save_folder, label, image_filename)
    
    # 이미 해당 경로에 이미지가 있다면, 저장하지 않고 넘어갑니다.
    if os.path.exists(image_path):
        continue

    # 1채널 16비트 PNG 이미지 생성
    image_data = X_test_reshaped[i]
    image_data = (image_data * 65535).astype('uint16')  # 16비트로 변환
    image = Image.fromarray(image_data, 'I;16')  # 16비트 단일 채널 이미지로 변환
    
    image.save(image_path)
    
print("Test 이미지 변환 및 저장이 완료되었습니다.")

Test 이미지 변환 및 저장이 완료되었습니다.


In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_model(input_shape, num_classes):
    model = models.Sequential()

    # Convolution 1
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.BatchNormalization())

    # Maxpooling 1
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Convolution 2
    model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(layers.BatchNormalization())

    # Maxpooling 2
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Convolution 3
    model.add(layers.Conv2D(256, kernel_size=(3, 3), activation='relu'))
    model.add(layers.BatchNormalization())

    # Maxpooling 3
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Flatten
    model.add(layers.Flatten())

    # FC 1
    model.add(layers.Dense(2048, activation='relu'))
    model.add(layers.Dropout(0.5))

    # FC 2
    model.add(layers.Dense(2048, activation='relu'))
    model.add(layers.Dropout(0.5))

    # FC 3 (출력층)
    model.add(layers.Dense(num_classes, activation='sigmoid'))

    # 모델 컴파일
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'AUC'])
    return model

In [5]:
import os
import numpy as np
import sklearn.metrics as metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 이미지 데이터 폴더 경로
train_data_folder = r'C:\Users\pc\Desktop\CNN\Train_Image'
val_data_folder = r'C:\Users\pc\Desktop\CNN\Validation_Image'
test_data_folder = r'C:\Users\pc\Desktop\CNN\Test_Image'

# 모델 생성
input_shape = (28, 28, 1)  # 입력 이미지의 크기와 채널 수
num_classes = 1  # 이진 분류 문제의 경우 클래스 개수는 1
model = create_model(input_shape, num_classes)

# ImageDataGenerator 생성
datagen = ImageDataGenerator(rescale=1./255)

# 학습 데이터 로드
train_generator = datagen.flow_from_directory(
    train_data_folder,
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=32,
    shuffle=True
)

# Validation 데이터 로드
val_generator = datagen.flow_from_directory(
    val_data_folder,
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=32,
    shuffle=False  # Validation 데이터는 섞지 않음
)

# Test 데이터 로드
test_generator = datagen.flow_from_directory(
    test_data_folder,
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=32,
    shuffle=False  # Test 데이터는 섞지 않음
)

# 모델 학습
epochs = 50
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)

# 모델 평가
test_loss, test_accuracy, test_auc = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
print("Test AUC:", test_auc)

Found 200000 images belonging to 2 classes.
Found 24000 images belonging to 2 classes.
Found 24000 images belonging to 2 classes.
Epoch 1/50
 648/6250 [==>...........................] - ETA: 7:42 - loss: 0.5384 - accuracy: 0.7971 - auc: 0.5080

KeyboardInterrupt: 